In [ ]:
# # https://machinelearningmastery.com/how-to-perform-object-detection-in-photographs-with-mask-r-cnn-in-keras/
# # https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/
# # https://www.pyimagesearch.com/2018/11/19/mask-r-cnn-with-opencv/
# # https://towardsdatascience.com/webcam-object-detection-with-mask-r-cnn-on-google-colab-b3b012053ed1

# tensorflow_version 1.x


# #change add the Mask_RCNN directory to files. 
# # example: replace 'mrcnn' with:
# # Mask_RCNN.mrcnn

# # install mask_rcnn library or it's better to upload it, with the custom files.
# !git clone https://github.com/matterport/Mask_RCNN.git

# #istall some libraries from tpu
# #!git clone https://github.com/tensorflow/tpu/

# # import libraries required for the mask_rcnn library
# !pip3 install -r 'Mask_RCNN/requirements.txt'
# !cd Mask_RCNN ; python setup.py install

# # download the Microsoft-coco dataset, in the mask_rcnn folder 
# !cd Mask_RCNN; wget 'https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5'

# # download the an image of an elephant
# !wget 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/03/elephant.jpg'

# #download an image of a car
# !wget 'https://i.insider.com/5d9b5bff52887931e8497a36'

# # kangaroo dataset
# !git clone https://github.com/experiencor/kangaroo.git
# !cd

# !unzip '/content/drive/My Drive/machinelearningdataset/DeepFashion2.zip'
# print('')
# print('')
# print ('###################################all done###################################')

In [ ]:
# # unzip deepfashion2 
# # there's a file download limit for files from google drive. switch between the shared files from
# # deepfashion2 and the files saved on the drive

# #shared deepfashion2 dataset.
# #!unzip -P 2019Deepfashion2** '/content/drive/My Drive/DeepFashion2 Dataset/json_for_validation.zip'

# #  deepfashion2 dataset saved on google drive
# !unzip -P 2019Deepfashion2** '/content/drive/My Drive/machinelearningdataset/json_for_validation.zip'
# print('\n','##########################done upzipping files.###########################################')
# !unzip -P 2019Deepfashion2** '/content/drive/My Drive/machinelearningdataset/test.zip'
# print('\n','##########################done upzipping test files.###########################################')
# !unzip -P 2019Deepfashion2** '/content/drive/My Drive/machinelearningdataset/train.zip'
# print('\n','##########################done upzipping train files.###########################################')
# !unzip -P 2019Deepfashion2** '/content/drive/My Drive/machinelearningdataset/validation.zip'
# print('\n','##########################done upzipping validation files.###########################################')


In [ ]:
# create a virtual environment to run older version of tensorflow
# mask rcnn runs on tf version 1.x

# running venv on jupyter
!python3 -m pip install --upgrade pip
!python3 -m venv virtual_fashion
!source virtual_fashion/bin/activate
# !ipython kernel install --user --navirtual_fashion



In [ ]:
# running venv on jupyter
!pip3 list


#double cheking tf version. inside the venv, tf==1.15, outside, it's tf>2.0
!python3 -c 'import tensorflow as tf; print(tf.__version__)'


In [ ]:
# setups related to  mask rcnn 
!pip3 install -r 'Mask_RCNN/requirements.txt'
!cd Mask_RCNN ; python3 setup.py install
# making sure that everything got installed properly
!pip3 show mask-rcnn

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# convert the annonations to coco-compatible

# upload the custom files (visualize.py and model.py) from 7gate_folder>Mask_RCNN>mrcnn
# replace the deepfashion2_to_coco.py file in the DeepFashion2>evalation. from 7gate folder
import os
import json
import matplotlib.pyplot as plt


img_folder_path = 'train/image'
dirListing = os.listdir(img_folder_path)

num_images= len(dirListing)
print(num_images)
!python DeepFashion2/evaluation/deepfashion2_to_coco.py

In [ ]:
from os import listdir
import numpy as np
import json

images_dir =  'train/image/'
annotations_dir = 'train/annos/'
count_index=0
inner_count=0
b_box=[]
segmentationssss=list()
category_id=list()
width=list()
height=[]
for filename in listdir(images_dir):
  count_index+=1
  image_id = filename[:-4]
  direecewdew= 'train/annos/' + image_id +'.json'
  with open(direecewdew) as f:
    data = f.read()
    jsondata = json.loads(data)
    for number in range(1,10):
      item_number='item'+ str(number)
      #print(item_number)
      try:
        b_box_iterate=jsondata.get(item_number)['bounding_box']
        b_box.append(b_box_iterate)
        segment=jsondata.get(item_number)['segmentation']
        segmentationssss.append(segment)
        category=jsondata.get(item_number)['category_id']
        category_id.append(category)
        w=b_box_iterate[2]
        width.append(w)
        h=b_box_iterate[3]
        height.append(h)
        img_path = images_dir + filename
        ann_path = annotations_dir + image_id + '.json'
        #self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
        #print(inner_count)
        inner_count=+1
        #print(inner_count)
        #print(item_number)  
      except Exception:
        pass

height=np.array(height)
width=np.array(width)
b_box=np.array(b_box)
height=min(height)
width=min(width)
# print(count_index)
# print(b_box)
# print(height)
# print(width)
# print(segmentationssss)
# print(category_id)
# print(inner_count)

print(len(b_box))
print(height)
print(int(np.mean(width)))

#try it now.
# there should be no more list eror in masks=zeroes[]

In [ ]:
from os import listdir
import json
import numpy as np
from numpy import zeros
from numpy import asarray
from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.config import Config
from Mask_RCNN.mrcnn.model import MaskRCNN


# class that defines and loads the kangaroo dataset
class FashionDataset(Dataset):
  def load_dataset(self, dataset_dir, is_train=True):
    self.add_class("dataset", 1, "fashion")
    images_dir =  'train/image/'
    annotations_dir = 'train/annos/'
    count_index=0
    b_box=[]
    segmentationssss=list()
    category_id=list()
    width=[]
    height=[]
    for filename in listdir(images_dir):
      count_index+=1
      image_id = filename[:-4]
      direecewdew= 'train/annos/' + image_id +'.json'
      with open(direecewdew) as f:
        data = f.read()
        jsondata = json.loads(data)
        for number in range(1,10):
          item_number='item'+ str(number)
          try:
            b_box_iterate=jsondata.get(item_number)['bounding_box']
            b_box.append(b_box_iterate)
            segment=jsondata.get(item_number)['segmentation']
            segmentationssss.append(segment)
            category=jsondata.get(item_number)['category_id']
            category_id.append(category)
            w=b_box_iterate[2]
            width.append(w)
            h=b_box_iterate[3]
            height.append(h)
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.json'
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
          except Exception:
            pass
    
    height=np.array(height)
    width=np.array(width)
    b_box=np.array(b_box)
    height=max(height)
    width=max(width)

    print(len(b_box))
    print(height)
    print(int(np.mean(width)))
    return b_box, height, width , segmentationssss, category_id 
  

  def load_mask(self, image_id):
    info = self.image_info[image_id]
    path = info['annotation']
    b_box, segmentationssss, category_id, height, width = self.load_dataset(path)

    masks = zeros([height, width, len(b_box)], dtype='uint8')
    class_ids = list()

    for i in range(len(b_box)):
      box = b_box[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('fashion'))
    return masks, asarray(class_ids, dtype='int32')

  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']

print('##')
# !ls
# !pip list

# this is the original path 
# "python.pythonPath": "/usr/local/opt/python/bin/python3.7",
# need to set it back after deactivation




In [ ]:
# define a configuration for the model
class FashionConfig(Config):
	# define the name of the configuration
	NAME = "fashion_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 1
	# number of training steps per epoch
	STEPS_PER_EPOCH = 131

# prepare train set
train_set = FashionDataset()
train_set.load_dataset('fashion', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = FashionDataset()
test_set.load_dataset('fashion', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = FashionConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers

model.load_weights('Mask_RCNN/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# train weights (output layers or 'heads')
# try:
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')
# except OSError:
#   print('error')
#   pass

In [ ]:
# this error for running tf 2.0
# it should be running tf 1.15 instead, because thats the version in the venv

#somehow check the version
# or pip install the version inside the python files

In [ ]:
train_set = FashionDataset()
train_set.load_dataset('fashion', is_train=True)

In [ ]:
print(len(train_set.load_dataset('fashion', is_train=True)))
print(len(test_set.load_dataset('fashion', is_train=True)))